In [ ]:
import torch

import copy
import numpy as np
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics
from torch import nn, optim

import torch.nn.functional as F

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
device

device(type='cuda')

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/DACON/train_features.csv')

In [ ]:
target = pd.read_csv('/content/drive/MyDrive/DACON/train_labels.csv')
train_labels = target

In [ ]:
test = pd.read_csv('/content/drive/MyDrive/DACON/test_features.csv')
submission = pd.read_csv('/content/drive/MyDrive/DACON/sample_submission.csv')

In [ ]:
train

,id,time,acc_x,acc_y,acc_z,gy_x,gy_y,gy_z
0,0,0,1.206087,-0.179371,-0.148447,-0.591608,-30.549010,-31.676112
1,0,1,1.287696,-0.198974,-0.182444,0.303100,-39.139103,-24.927216
2,0,2,1.304609,-0.195114,-0.253382,-3.617278,-44.122565,-25.019629
3,0,3,1.293095,-0.230366,-0.215210,2.712986,-53.597843,-27.454013
4,0,4,1.300887,-0.187757,-0.222523,4.286707,-57.906561,-27.961234
...,...,...,...,...,...,...,...,...
1874995,3124,595,-0.712530,-0.658357,0.293707,-29.367857,-104.013664,-76.290437
1874996,3124,596,-0.683037,-0.658466,0.329223,-30.149089,-101.796809,-76.625087
1874997,3124,597,-0.664730,-0.666625,0.364114,-27.873095,-98.776072,-79.365125
1874998,3124,598,-0.630534,-0.682565,0.373696,-23.636550,-99.139495,-80.259478


In [ ]:
train_ = train.copy()

In [ ]:
# train_ 데이터프레임에 group 컬럼을 추가한다.

train_['group'] = 1

#train_['group'] = np.where(train_['time'] <= 600 , 1, 0)

In [ ]:
for i in range(59 , -1 , -1):
  train_.loc[(train_.time < (i+1)*10), 'group'] = i
  
# 0 ~ 9
# 10 ~ 19
# 20 ~ 29
# 30 ~ 39
# ...
# 590 ~ 599

In [ ]:
group_train_ = train_.groupby(['id', 'group'])

In [ ]:
df =  pd.DataFrame(group_train_.mean())

In [ ]:
df.drop(['time'], axis = 1)

acc_x     acc_y     acc_z       gy_x        gy_y       gy_z
id   group                                                                
0    0      1.251791 -0.229154 -0.232241   5.313051  -59.202390 -29.109459
     1      1.068298 -0.197191 -0.269734   5.586739  -42.667238 -16.549480
     2      0.722119 -0.114564 -0.339607 -11.819742    6.212027  18.009191
     3      0.679590  0.016110 -0.555735  -3.625908    4.789341  11.917563
     4      0.742513 -0.027839 -0.583247 -16.800999  -12.506986   3.754061
...              ...       ...       ...        ...         ...        ...
3124 55    -0.961550 -0.317255 -0.275331   8.033097   56.305529  20.814754
     56    -0.969101 -0.290030 -0.259020  -1.682063    2.838185   1.094412
     57    -0.971646 -0.339630 -0.166837  -9.983776  -59.722361 -25.377563
     58    -0.929038 -0.440118  0.049172  -8.414001  -94.243835 -55.653302
     59    -0.725891 -0.644950  0.288444 -23.531633 -101.818319 -75.885761

[187500 rows x 6 columns]

In [ ]:
df = df.drop(['time'], axis = 1)

In [ ]:
df.shape

(187500, 6)

In [ ]:
X = torch.FloatTensor(df.values).view(-1, 60, 6)

In [ ]:
X.shape

torch.Size([3125, 60, 6])

In [ ]:
train_labels

,id,label,label_desc
0,0,37,Shoulder Press (dumbbell)
1,1,26,Non-Exercise
2,2,3,Biceps Curl (band)
3,3,26,Non-Exercise
4,4,26,Non-Exercise
...,...,...,...
3120,3120,26,Non-Exercise
3121,3121,26,Non-Exercise
3122,3122,15,Dynamic Stretch (at your own pace)
3123,3123,26,Non-Exercise


In [ ]:
y = torch.FloatTensor(train_labels['label'].values)

In [ ]:
y.shape

torch.Size([3125])

In [ ]:
def to_categorical(y, num_classes):
    """ 1-hot encodes a tensor """
    return np.eye(num_classes, dtype='uint8')[y]

In [ ]:
y = y.view(-1, 1)

In [ ]:
y.shape

torch.Size([3125, 1])

In [ ]:
class Net(torch.nn.Module):
  def __init__(self , input_size , hidden_size):
    super(Net, self).__init__()
    self.lstm = torch.nn.LSTM(input_size , hidden_size , batch_first= True)
    self.fc1 = torch.nn.Linear(hidden_size , 128 , bias = True)
    self.relu = torch.nn.ReLU()
    self.fc2 = torch.nn.Linear(128,61 ,bias = True)

  def forward(self, x):
    x, _status = self.lstm(x)
    x = self.fc1(x)
    x = self.relu(x)
    x = self.fc2(x)
    return x

In [ ]:
output = Net(6,61)

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(output.parameters(), 1e-3)

In [ ]:
y = y.squeeze(1).long()

In [ ]:
for i in range(400):
  optimizer.zero_grad()
  outputs = output(X)
  loss = criterion(outputs[:,-1,:] , y)
  loss.backward()
  optimizer.step()
  
  train_acc = torch.sum(torch.argmax(outputs[:,-1,:], axis = 1) == y)

  print('epoch : ',i, " loss : ", loss.item())
  print('acc : ', (train_acc / outputs.shape[0]).numpy())

epoch :  0  loss :  4.121762275695801
acc :  0.0048
epoch :  1  loss :  4.096011638641357
acc :  0.00672
epoch :  2  loss :  4.070350646972656
acc :  0.01792
epoch :  3  loss :  4.044595718383789
acc :  0.06944
epoch :  4  loss :  4.018551349639893
acc :  0.19264
epoch :  5  loss :  3.9920475482940674
acc :  0.32544
epoch :  6  loss :  3.9648797512054443
acc :  0.41728
epoch :  7  loss :  3.9368491172790527
acc :  0.46432
epoch :  8  loss :  3.907749891281128
acc :  0.47936
epoch :  9  loss :  3.8773748874664307
acc :  0.48416
epoch :  10  loss :  3.8454935550689697
acc :  0.48576
epoch :  11  loss :  3.811861276626587
acc :  0.48576
epoch :  12  loss :  3.7761893272399902
acc :  0.48576
epoch :  13  loss :  3.7382662296295166
acc :  0.48576
epoch :  14  loss :  3.6978566646575928
acc :  0.48576
epoch :  15  loss :  3.6546783447265625
acc :  0.48576
epoch :  16  loss :  3.608474016189575
acc :  0.48576
epoch :  17  loss :  3.558885097503662
acc :  0.48576
epoch :  18  loss :  3.5056481

In [ ]:
outputs.shape[0]

3125

In [ ]:
with torch.no_grad():
  #X_test = torch.FloatTensor(test.iloc[:,2:].values).view(-1, 600, 6)
  pred_test = output(X)

In [ ]:
x_numpy = pred_test.numpy()

In [ ]:
x_numpy = x_numpy[:, -1, :]

In [ ]:
x_numpy = np.argmax(x_numpy, axis = 1)

In [ ]:
x_numpy

array([37, 26,  3, ..., 15, 26,  2])

In [ ]:
y_numpy = y.numpy()
y_numpy

array([37, 26,  3, ..., 15, 26,  2])

In [ ]:
import sklearn.metrics as metrics

In [ ]:
acc = metrics.accuracy_score(x_numpy , y_numpy)
print('acc : ', acc)

acc :  0.94208


In [ ]:
pd.DataFrame(torch.argmax(pred_test[:, -1, :], axis = 1))

,0
0,37
1,26
2,3
3,26
4,26
...,...
3120,26
3121,26
3122,15
3123,26


In [ ]:
test_df = pd.DataFrame(pred_test[:, -1, :].numpy())

In [ ]:
test_df.to_excel('/content/drive/MyDrive/DACON/excel_test.xlsx의 사본.xlsx')

In [ ]:
torch.save(output.state_dict(), '/content/drive/MyDrive/DACON/data/EMS_LSTM_epoch_400')